In [ ]:
# Model the cost of TBB's shared map
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math
import numpy as np
from scipy import optimize
import sys

figsize=(4.5, 3)

In [ ]:
# Read the data
tbb = pd.read_csv("tbbSize.csv")
for _,r in tbb.iterrows():
    r['add'] /= r['size']

In [ ]:
# Define function and plot

tbbMean = tbb.groupby(['size'])['add'].mean()
steps = tbb['size'].unique()

def fitfunc(x,t,ox,oy):
    def runner(a):
        #return (a+ox)*t + oy
        oxnow = ox
        if (a - ox) < 1:
            oxnow = a - 1
        return math.log((a-oxnow),t) + oy
        #return math.log(a, t) +oy
    if isinstance(x, np.ndarray):
        ret = []
        for s in x:
            ret += [runner(s)]
        return ret
    else:
        return runner(x)

params = [1.0002,2**15,600]
params, _ = optimize.curve_fit(fitfunc, steps, tbbMean, p0=params, bounds=([1.0001, 0, 0], [1.03, 10000000, 800]))

fit = []
for s in steps:
    fit += [fitfunc(s,params[0],0,params[2])]
    
print(params)


smalls = []
for x in [1,2,4,8,16,32]:
    smalls += [(x, fitfunc(x,params[0],params[1],params[2]))]

for s in smalls:
    print(s)

fig = plt.figure(figsize=figsize, dpi=300)
plt.plot(steps, fit, label="fit")
plt.plot(steps, tbbMean, label="data")
plt.ylim(0,1600)
plt.ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.legend()
plt.ylabel("Cycles per Operation")
plt.xlabel("#Operations")
plt.grid(b=True)
plt.savefig('output/synth-tbb.pdf',bbox_inches='tight')
plt.show()

fig = plt.figure(figsize=figsize, dpi=300)
plt.semilogx(basex=2)
plt.plot(steps, fit, label="fit")
plt.plot(steps, tbbMean, label="data")
plt.ylim(0,1600)
plt.grid(b=True)
plt.legend()
plt.show()